## Imports

In [1]:
import io
import os.path
from scrapy import Selector
from datetime import datetime
from unicodedata import normalize
from urllib.request import Request, urlopen

In [2]:
papel = "PETR4"
url = 'https://fundamentus.com.br/detalhes.php?papel=%s' % papel

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

response = urlopen(req, timeout=20).read()
html_dados = response.decode('latin-1')
    
global html_selector
html_selector = Selector( text = html_dados )

In [3]:
len(html_selector.xpath("//table[@class='w728']").extract())

5

In [4]:
str(type(html_selector)) == "<class 'scrapy.selector.unified.Selector'>"

True

In [5]:
len(html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[0::2][0::2])

5

In [5]:
html_selector.xpath("//table[1]").extract() 

['<table class="w728">\n<tr>\n<td class="label w15"><span class="help tips" title="Código da ação">?</span><span class="txt">Papel</span></td>\n<td class="data w35"><span class="txt">PETR4</span></td>\n<td class="label destaque w2"><span class="help tips" title="Cotação de fechamento da ação no último pregão">?</span><span class="txt">Cotação</span></td>\n<td class="data destaque w3"><span class="txt">23,46</span></td>\n</tr>\n<tr>\n<td class="label"><span class="help tips" title="ON = Ordinária, PN = Preferencial, PNA = Pref. tipo A, etc">?</span><span class="txt">Tipo</span></td>\n<td class="data"><span class="txt">PN</span></td>\n<td class="label"><span class="help tips" title="Data do último pregão em  que o ativo foi negociado">?</span><span class="txt">Data últ cot</span></td>\n<td class="data"><span class="txt">26/03/2021</span></td>\n</tr>\n<tr>\n<td class="label"><span class="help tips" title="Nome comercial da empresa.">?</span><span class="txt">Empresa</span></td>\n<td class

In [6]:
info_empresa_label = html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[0::2][0::2]
info_empresa_label = remover_caracteres_especiais(info_empresa_label)
print(info_empresa_label)

NameError: name 'remover_caracteres_especiais' is not defined

## Baixando HTML do site Fundamentus (Web Crawling)

In [4]:
def baixar_html_site_fundamentus(papel):
    print(f"Baixando dados em HTML do '{papel}' no site fundamentus...")
    print("--------------------------------------------")
    url = 'https://fundamentus.com.br/detalhes.php?papel=%s' % papel

    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    response = urlopen(req, timeout=20).read()
    html_dados = response.decode('latin-1')
    
    global html_selector
    html_selector = Selector( text = html_dados )

## Funções para transformar os dados e labels

In [5]:
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

#if __name__ == '__main__':
#    from doctest import testmod
#    testmod()

In [6]:
def remover_caracteres_especiais(empresa_label):
    empresa_label = list(map(lambda x: remover_acentos(x), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, " ","_"), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, "(",""), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, ")",""), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, ".",""), empresa_label))
    return empresa_label

In [7]:
def validar_data(valor):
    try:
        valor = datetime.strptime(valor, '%d/%m/%Y')
        return valor
    except TypeError:
        return valor

In [8]:
def formatar_tipagem_dados(empresa_dados):
    empresa_dados = list(map(lambda st: str.replace(st, ",","."), empresa_dados))
    
    nova_lista_empresa_dados = []
    for valor in empresa_dados:
        eh_valor_percentual = valor.endswith('%')
        
        if eh_valor_percentual:
            nova_lista_empresa_dados.append(valor)
        elif "/" in valor:
            nova_lista_empresa_dados.append(validar_data(valor))
        elif not eh_valor_percentual:
            try:
                nova_lista_empresa_dados.append(float(valor))
            except ValueError:
                nova_lista_empresa_dados.append(valor)
    
    return nova_lista_empresa_dados

In [9]:
def transformar_lista_em_tuples(empresa_dados):
    empresa_dados_list = []
    empresa_dados_list.append(tuple(empresa_dados))
    return empresa_dados_list

In [10]:
def transformar_listas_em_dicionario(lista_keys, lista_values):
    novo_dicionario = {lista_keys[i]: lista_values[i] for i in range(len(lista_keys))}
    
    return novo_dicionario

In [11]:
transformar_lista_em_tuples([1,2,3,4])

[(1, 2, 3, 4)]

## Pegando data última cotação e data último balanço

In [12]:
def extrair_data_ult_cotacao():
    data_ult_cotacao = html_selector.xpath("//table[1]//span[@class='txt']/text() | // table[1]//span[@class='txt']/a/text()").extract()[7]
    #data_ult_cotacao = validar_data(data_ult_cotacao)
    return data_ult_cotacao

def extrair_data_ult_balanco():
    data_ult_balanco = html_selector.xpath("//table[2]//span[@class='txt']/text()").extract()[3]
    #data_ult_balanco = validar_data(data_ult_balanco)
    return data_ult_balanco

In [15]:
extrair_data_ult_cotacao()

'05/04/2021'

## Verificação se já existe os dados:
* Verificar se o papel já existe nos dados gravados
    * Se não existir, inserir dados da empresa, dados de cotação e dados do balanço
    * Se existir, verificar se a cotação mais atual já existe
    * Se existir, verificar se o balanço mais atual já existe
* Verificando se a cotaçao mais atual já existe
    * Se não existir, inserir dados da cotação atualizado
    * Se existir, não fazer nada
* Verificando se o balanço mais atual já existe
    * Se não existir, inserir dados do balanço atualizado
    * Se existir, não fazer nada

## Conexão com o MongoDB

In [47]:
from pymongo import MongoClient

def conexao_banco_mongodb():    
    cliente = MongoClient('mongodb://localhost:27017/')
    global conexao_banco_mongodb
    conexao_banco_mongodb = cliente['test_fundamentus']

conexao_banco_mongodb()

In [52]:
str(type(conexao_banco_mongodb.info_empres))

"<class 'pymongo.collection.Collection'>"

### Buscar dados no MongoDB

In [8]:
def buscar_dados_empresa_por_papel(papel):
    colecao = conexao_banco_mongodb.info_empresa
    dados_empresa = colecao.find_one({"Papel": papel})
    
    return dados_empresa

In [10]:
dados = buscar_dados_empresa_por_papel('PETR')
dados

In [13]:
str(type(dados))

"<class 'NoneType'>"

In [16]:
def buscar_lista_cotacoes_empresa_por_papel(papel):
    colecao = conexao_banco_mongodb.cotacao
    lista_cotacoes = colecao.find({"Papel": papel})
    
    return lista_cotacoes

In [17]:
def buscar_cotacao_empresa_por_papel_data(papel, data_cotacao):
    colecao = conexao_banco_mongodb.cotacao
    cotacao_papel_data = colecao.find_one({"Papel": papel, "Data_ult_cot": data_cotacao})
    
    return cotacao_papel_data

In [18]:
def buscar_lista_balancos_empresa_por_papel(papel):
    colecao = conexao_banco_mongodb.balanco
    lista_balancos = colecao.find({"Papel": papel})
    
    return lista_balancos

In [19]:
def buscar_balanco_empresa_por_papel_data(papel, data_balanco):
    colecao = conexao_banco_mongodb.balanco
    balanco_papel_data = colecao.find_one({"Papel": papel, "Data_ult_cot": data_balanco})
    
    return balanco_papel_data

### Atualizar dados no MongoDB

In [20]:
def atualizar_oscilacao_na_cotacao_mongodb(id_inserido_oscilacoes, id_inserido_cotacao):
    colecao = conexao_banco_mongodb.cotacao
    colecao.update_one({"_id":  id_inserido_cotacao}, {'$set': {'Oscilacao':id_inserido_oscilacoes}})

In [21]:
def atualizar_indicadores_fundamentalistas_na_cotacao_mongodb(id_inserido_indicadores, id_inserido_cotacao):
    colecao = conexao_banco_mongodb.cotacao
    colecao.update_one({"_id":  id_inserido_cotacao}, {'$set': {'Indicador_Fundamentalista': id_inserido_indicadores}})

In [22]:
# Balanço da Empresa
def atualizar_cotacoes_no_balanco_por_id_mongodb(id_inserido_balanco, id_inserido_cotacao):
    colecao = conexao_banco_mongodb.balanco
    colecao.update_one({"_id":  id_inserido_balanco}, {'$push': {'Cotacoes': id_inserido_cotacao }})

In [23]:
def atualizar_cotacoes_no_balanco_por_papel_data_mongodb(papel, data_ult_balanco, id_inserido_cotacao):
    print("*** Atualizando dados de Cotação no Balanço atual da Empresa...")
    colecao = conexao_banco_mongodb.balanco
    colecao.update_one({"Papel":  papel, "Ult_balanco_processado": data_ult_balanco}, {'$push': {'Cotacoes': id_inserido_cotacao }})

### Inserir dados no MongoDB

In [24]:
# Dados da Empresa
def inserir_dados_empresa_mongodb(info_empresa):
    print("** Salvando Informações da Empresa...")
    colecao = conexao_banco_mongodb.dados_empresa
    colecao.insert_one(info_empresa)

In [25]:
# Cotação da Empresa
def inserir_cotacao_empresa_mongodb(cotacao_empresa):
    print("** Salvando dados de Cotação da Empresa...")
    colecao = conexao_banco_mongodb.cotacao
    id_inserido_cotacao = colecao.insert_one(cotacao_empresa).inserted_id
    
    return id_inserido_cotacao

In [26]:
# Oscilações da Empresa
def inserir_oscilacoes_empresa_mongodb(oscilacoes_empresa, id_inserido_cotacao):
    print("** Salvando dados de Oscilações da Empresa...")
    oscilacoes_empresa['Cotacao'] = id_inserido_cotacao
    colecao = conexao_banco_mongodb.oscilacoes
    id_inserido_oscilacoes = colecao.insert_one(oscilacoes_empresa).inserted_id
    
    atualizar_oscilacao_na_cotacao_mongodb(id_inserido_oscilacoes, id_inserido_cotacao)

In [27]:
# Indicadores Fundamentalistas da Empresa
def inserir_indicadores_fundamentalista_empresa_mongodb(indicadores_fundamentalitas_empresa, id_inserido_cotacao):
    print("** Salvando dados de Indicadores Fundamentalistas da Empresa...")
    indicadores_fundamentalitas_empresa['Cotacao'] = id_inserido_cotacao
    colecao = conexao_banco_mongodb.indicadores_fundamentalista
    id_inserido_indicadores = colecao.insert_one(indicadores_fundamentalitas_empresa).inserted_id

    atualizar_indicadores_fundamentalistas_na_cotacao_mongodb(id_inserido_indicadores, id_inserido_cotacao)

In [28]:
def inserir_balanco_empresa_mongodb(balanco_empresa, id_inserido_cotacao):
    print("** Salvando dados de Balanço da Empresa...")
    colecao = conexao_banco_mongodb.balanco
    id_inserido_balanco = colecao.insert_one(balanco_empresa).inserted_id
    
    atualizar_cotacoes_no_balanco_por_id_mongodb(id_inserido_balanco, id_inserido_cotacao)

## Extrair Dados da Empresa e transformar em dicionario

In [13]:
def extrair_info_empresa_label():
    info_empresa_label = html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[0::2][0::2]
    info_empresa_label = remover_caracteres_especiais(info_empresa_label)
    
    return info_empresa_label

In [14]:
def extrair_info_empresa_dados():
    info_empresa_dados = html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[1::2][0::2]
    
    return info_empresa_dados

In [15]:
def criar_dicionario_info_empresa():
    print('* Extraindo Informações da Empresa...')
    info_empresa_label = extrair_info_empresa_label()
    info_empresa_dados = extrair_info_empresa_dados()
    info_empresa = transformar_listas_em_dicionario(info_empresa_label, info_empresa_dados)
    
    return info_empresa

## Extraindo Dados atualizados de Cotação (caso estejam desatualizados)
### Extrair Dados Básicos Cotação e transformar em dicionario

In [16]:
def extrair_cotacao_label():
    cotacao_label = html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[0::2][1::2]
    cotacao_label += html_selector.xpath("//table[2]//span[@class='txt']/text()").extract()[0::2][0::3]
    cotacao_label = remover_caracteres_especiais(cotacao_label)
    cotacao_label.insert(0, 'Papel')    

    return cotacao_label

In [17]:
def extrair_cotacao_dados(papel):
    cotacao_dados = html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[1::2][1::2]
    cotacao_dados += html_selector.xpath("//table[2]//span[@class='txt']/text()").extract()[1::2][0::3]
    #cotacao_dados = formatar_tipagem_dados(cotacao_dados)
    cotacao_dados.insert(0, papel)
    
    return cotacao_dados

In [18]:
def criar_dicionario_cotacao_empresa(papel):
    print('* Extraindo Cotação da Empresa...')
    cotacao_label = extrair_cotacao_label()
    cotacao_dados = extrair_cotacao_dados(papel)
    cotacao_empresa = transformar_listas_em_dicionario(cotacao_label, cotacao_dados)
    
    return cotacao_empresa


### Extrair Dados Oscilação e transformar em dicionario

In [19]:
def extrair_oscilacao_cotacao_label():
    oscilacao_label = html_selector.xpath("//table[3]//td[@class='nivel1']//span/text()").extract()[0]
    lista_oscilacoes_label = html_selector.xpath("//table[3]//td[@class='label w1']//span/text()").extract()
    oscilacoes_label = list(map(lambda label: oscilacao_label + "_" + label, lista_oscilacoes_label))
    oscilacoes_label = remover_caracteres_especiais(oscilacoes_label)
    oscilacoes_label.insert(0, 'Data_ult_cot')
    oscilacoes_label.insert(0, 'Papel')
    
    return oscilacoes_label

In [20]:
def extrair_oscilacao_cotacao_dados(papel):
    oscilacoes_dados = html_selector.xpath("//table[3]//span[@class='oscil']/font/text()").extract()
    oscilacoes_dados.insert(0, extrair_data_ult_cotacao())
    oscilacoes_dados.insert(0, papel)
    #oscilacoes_dados = formatar_tipagem_dados(oscilacoes_dados)
    
    return oscilacoes_dados

In [21]:
def criar_dicionario_oscilacoes_empresa(papel):
    print('* Extraindo Oscilações de Cotação da Empresa...')
    oscilacoes_label = extrair_oscilacao_cotacao_label()
    oscilacoes_dados = extrair_oscilacao_cotacao_dados(papel)
    oscilacoes_empresa = transformar_listas_em_dicionario(oscilacoes_label, oscilacoes_dados)
    
    return oscilacoes_empresa

### Extrair Dados Indicadores Fundamentalistas e transformar em dicionario

In [22]:
def extrair_indicadores_fundamentalistas_label():
    indicador_fundamentalita_label = html_selector.xpath("//table[3]//td[@class='label w2']//span[@class='txt']/text() | //table[3]//td[@class='label']//span[@class='txt']/text()").extract()
    indicador_fundamentalita_label = remover_caracteres_especiais(indicador_fundamentalita_label)
    indicador_fundamentalita_label.insert(0, 'Data_ult_cot')
    indicador_fundamentalita_label.insert(0, 'Papel')
    
    return indicador_fundamentalita_label

In [23]:
def extrair_indicadores_fundamentalistas_dados(papel):
    indicador_fundamentalita_dados = html_selector.xpath("//table[3]//td[@class='data w2']//span[@class='txt']/text() | //table[3]//td[@class='data']//span[@class='txt']/text()").extract()
    indicador_fundamentalita_dados.insert(0, extrair_data_ult_cotacao())
    indicador_fundamentalita_dados.insert(0, papel)
    indicador_fundamentalita_dados = list(map(lambda dado: dado.replace("\n", ""), indicador_fundamentalita_dados))
    #indicador_fundamentalita_dados = formatar_tipagem_dados(indicador_fundamentalita_dados)
    
    return indicador_fundamentalita_dados

In [24]:
def criar_dicionario_indicadores_fundamentalistas_empresa(papel):
    print('* Extraindo Indicadores Fundamentalistas da Empresa...')
    indicador_fundamentalita_label = extrair_indicadores_fundamentalistas_label()
    indicador_fundamentalita_dados = extrair_indicadores_fundamentalistas_dados(papel)
    indicadores_fundamentalistas_empresa = transformar_listas_em_dicionario(indicador_fundamentalita_label, indicador_fundamentalita_dados)

    return indicadores_fundamentalistas_empresa

## Extrair Dados do Balanço e transformar em dicionario

In [25]:
def extrair_balanco_label():
    balanco_label = html_selector.xpath("//table[2]//span[@class='txt']/text()").extract()[2:6][0::2]
    balanco_label += html_selector.xpath("//table[4]//td[@class='label w2']//span[@class='txt']/text() | //table[4]//td[@class='label']//span[@class='txt']/text()").extract()
    dre_12_meses = html_selector.xpath("//table[5]//td[@class='label w2']//span[@class='txt']/text() | //table[5]//td[@class='label']//span[@class='txt']/text()").extract()[0::2]
    balanco_label += list(map(lambda label: label + "_ult._12_meses", dre_12_meses))
    dre_3_meses = html_selector.xpath("//table[5]//td[@class='label w2']//span[@class='txt']/text() | //table[5]//td[@class='label']//span[@class='txt']/text()").extract()[1::2]
    balanco_label += list(map(lambda label: label + "_ult._3_meses", dre_3_meses))
    balanco_label = remover_caracteres_especiais(balanco_label)
    balanco_label.insert(0, 'Papel')
    
    return balanco_label

In [26]:
def extrair_balanco_dados(papel):
    balanco_dados = html_selector.xpath("//table[2]//span[@class='txt']/text()").extract()[2:6][1::2]
    balanco_dados += html_selector.xpath("//table[4]//td[@class='data w3']//span[@class='txt']/text() | //table[4]//td[@class='data']//span[@class='txt']/text()").extract()
    balanco_dados += html_selector.xpath("//table[5]//td[@class='data w3']//span[@class='txt']/text() | //table[5]//td[@class='data']//span[@class='txt']/text()").extract()[0::2]
    balanco_dados += html_selector.xpath("//table[5]//td[@class='data w3']//span[@class='txt']/text() | //table[5]//td[@class='data']//span[@class='txt']/text()").extract()[1::2]
    balanco_dados.insert(0, papel)
    #balanco_dados = formatar_tipagem_dados(balanco_dados)
    
    return balanco_dados

In [27]:
def criar_dicionario_balanco_empresa(papel):
    print('* Extraindo dados do Balanço da Empresa...')
    balanco_label = extrair_balanco_label()
    balanco_dados = extrair_balanco_dados(papel)
    balanco_empresa = transformar_listas_em_dicionario(balanco_label, balanco_dados)
    
    return balanco_empresa

## Adicionar Dados a serem inseridos no MongoDB

In [44]:
def adicionar_info_empresa():
    info_empresa = criar_dicionario_info_empresa()
    inserir_dados_empresa_mongodb(info_empresa)

In [45]:
def adicionar_cotacao_empresa(papel):
    cotacao_empresa = criar_dicionario_cotacao_empresa(papel)    
    id_inserido_cotacao = inserir_cotacao_empresa_mongodb(cotacao_empresa)
        
    oscilacoes_empresa = criar_dicionario_oscilacoes_empresa(papel)
    inserir_oscilacoes_empresa_mongodb(oscilacoes_empresa, id_inserido_cotacao)
        
    indicadores_fundamentalistas_empresa = criar_dicionario_indicadores_fundamentalistas_empresa(papel)
    inserir_indicadores_fundamentalista_empresa_mongodb(indicadores_fundamentalistas_empresa, id_inserido_cotacao)
    
    return id_inserido_cotacao

In [46]:
def adicionar_balanco_empresa(papel, id_inserido_cotacao):
    balanco_empresa = criar_dicionario_balanco_empresa(papel)
    inserir_balanco_empresa_mongodb(balanco_empresa, id_inserido_cotacao)

### Verificação se dados já existem no MongoDB

In [47]:
def dados_empresa_nao_existem(papel):
    dados_empresa = buscar_dados_empresa_por_papel(papel)
    if dados_empresa is None:
        return True
    else:
        return False 
    

def ultima_cotacao_nao_existem(papel, data_ult_cotacao):
    ultima_cotacao = buscar_cotacao_empresa_por_papel_data(papel, data_ult_cotacao)
    if ultima_cotacao is None:
        return True
    else:
        return False
    

def ultimo_balanco_nao_existem(papel, data_ult_balanco):
    ultimo_balanco = buscar_balanco_empresa_por_papel_data(papel, data_ult_balanco)
    if ultimo_balanco is None:
        return True
    else:
        return False

## Método principal (main)

In [169]:
def metodo_main():
    
    print('Iniciando Web Scraping:')
    print("--------------------------------------------")
    papel = input("Digite o código do papel: ").upper()
    
    baixar_html_site_fundamentus(papel)
    data_ult_cotacao = extrair_data_ult_cotacao()
    data_ult_balanco = extrair_data_ult_balanco()
    
    # ADICIONAR DADOS DA EMPRESA
    if dados_empresa_nao_existem(papel):
        print(f"Dados da empresa '{papel}' ainda não existem:")
        adicionar_info_empresa()
        id_inserido_cotacao = adicionar_cotacao_empresa(papel)
        adicionar_balanco_empresa(papel, id_inserido_cotacao)

    # ADICIONAR DADOS DA ÚLTIMA COTAÇÃO
    # ADICIONAR DADOS DA ÚLTIMA OSCILAÇÃO
    # ADICIONAR DADOS DOS ÚLTIMOS INDICADORES FUNDAMENTALISTAS
    elif ultima_cotacao_nao_existem(papel, data_ult_cotacao):
        print(f"Dados da empresa '{papel}' já existem:")
        print('* Atualizando Dados de Cotação....')
        id_inserido_cotacao = adicionar_cotacao_empresa(papel)
        
        if ultimo_balanco_nao_existem(papel, data_ult_balanco):
            print("--------------------------------------------")
            print('* Adicionando Balanço atualizado da empresa....')
            adicionar_balanco_empresa(papel, id_inserido_cotacao)
        
        else:
            print("--------------------------------------------")
            print('Dados atuais do Balanço da empresa já existem:')
            print('* Adicionando Balanço atualizado da empresa....')
            atualizar_cotacoes_no_balanco_por_papel_data_mongodb(papel, data_ult_balanco, id_inserido_cotacao)
    else:
        print("--------------------------------------------")
        print('Todos os dados já estão atualizados!')

In [170]:
metodo_main()

Iniciando Web Scraping:
--------------------------------------------


Digite o código do papel:  GGBR4


Baixando dados em HTML do 'GGBR4' no site fundamentus...
--------------------------------------------
Dados da empresa 'GGBR4' ainda não existem:
* Extraindo Informações da Empresa...
** Salvando Informações da Empresa...
* Extraindo Cotação da Empresa...
** Salvando dados de Cotação da Empresa...
* Extraindo Oscilações de Cotação da Empresa...
** Salvando dados de Oscilações da Empresa...
* Extraindo Indicadores Fundamentalistas da Empresa...
** Salvando dados de Indicadores Fundamentalistas da Empresa...
* Extraindo dados do Balanço da Empresa...
** Salvando dados de Balanço da Empresa...


## Buscando dados no MongoDB

In [73]:
def buscar_balanco_por_papel(papel):
    colecao = conexao_banco_mongodb.balanco
    resultado = colecao.find_one({ "Papel": papel})
    
    return resultado

resultado = buscar_balanco_por_papel('ggbr4')
resultado

In [74]:
if (resultado is None) or (len(resultado) == 0):
    print('Vazio')
else:
    print(True)

Vazio


In [ ]:
def buscar_cotacao_e_dados_oscilacao(papel):
    pipeline = [
        {"$match" : { "Papel" : papel }},
        {"$lookup" : { "from": "oscilacoes",
                       "localField": "Papel",
                       "foreignField": "Papel",
                       "as": "Oscilacao_Papel" }},
        {"$unset": ["Oscilacao_Papel._id", "Oscilacao_Papel.Papel", "Oscilacao_Papel.Cotacao"]} # Ocultando campos no retorno
    ]
    colecao = conexao_banco_mongodb.cotacao
    resultado = list(colecao.aggregate(pipeline))
    
    return resultado

resultado = buscar_cotacao_e_dados_oscilacao("PETR4")
resultado

In [ ]:
resultado[0]['Oscilacao_Papel']

### Testando inclusão de item na lista Cotacoes na coleção Balanço

In [98]:
resultado = buscar_balanco_por_papel("PETR4")
id_balanco = resultado['_id']

atualizar_lista_cotacoes_no_balanco(id_balanco)

NameError: name 'buscar_balanco_por_papel' is not defined

# --------------------------------------------------------------------------------------------

## Verificando se o arquivo já existe

In [37]:
#path = '/home/hugo/Documents/Repositorios_GitHub/Projetos/02-StockData/Web_Scraping/Fundamentus_Web/'
#file_name = '%s_fundamentus_%s.txt' % (papel, str(data_ult_balanco_new_html))

#if os.path.isfile(path+file_name):
#    print('Arquivo já existe com balanço mais atual!')
#else:
#    print('Arquivo com balanço atual ainda não existe!')

## Salvando conteúdo do HTML (em bytes) em um arquivo local como txt

In [38]:
#file_name = '%s_fundamentus_%s.txt' % (papel, data_ult_balanco_new_html)

#with io.open(path+file_name, 'w', encoding='utf-8') as f:
#    f.write(new_html)

## Lendo conteúdo do arquivo Local e armazenando em uma variável

In [39]:
#with io.open(path+file_name, 'r', encoding='utf8') as f:
#    html = f.read()

# -----------------------------------------------------------------------------------------

In [53]:
# ADICIONA ITEM A UMA LISTA

#colecao = banco.cotacao
#colecao.update_one({"_id":  id_inserido_cotacao}, {'$push': {'oscilacoes': id_inserido_cotacao }})

In [59]:
#colecao = banco.balanco

#colecao.update_one({"_id":  id_inserido_balanco}, {'$set': {'Indicador_Fundamentalista': id_inserido_indicadores}})

# -----------------------------------------------------------------------------------------

# Acesso ao HDFS server (Write and Read)
## Criando uma Spark Session

In [40]:
#!pip3 install pyspark

In [43]:
from pyspark.sql import SparkSession

def criar_spark_session():
    global sparkSession
    sparkSession = SparkSession.builder.appName("write-read-fundamentus-hdfs").getOrCreate()
criar_spark_session()

NoneType

In [46]:
str(type(sparkSession))

"<class 'pyspark.sql.session.SparkSession'>"

## Write Spark DataFrame dos Dados da Empresa

### Necessário configurar a versão do Python utilizada pelo Linux como a 3.6.
* sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.4 1
* sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.6 2
* sudo update-alternatives --config python
* sudo update-alternatives  --set python /usr/bin/python3.6

In [45]:
empresa_dados

NameError: name 'empresa_dados' is not defined

In [29]:
def criar_spark_dataframe(empresa_label, empresa_dados):
    dados_tuple = transformar_lista_em_tuples(empresa_dados)
    empresa_df = sparkSession.createDataFrame(dados_tuple, empresa_label)
    
    return empresa_df

In [30]:
dataframe_label = extrair_info_empresa_label()
dataframe_dados = extrair_info_empresa_dados()
    
empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)

In [31]:
empresa_df

DataFrame[Papel: string, Tipo: string, Empresa: string, Setor: string, Subsetor: string]

In [55]:
dataframe_dados

['PETR4',
 'PN',
 'PETROBRAS PN',
 'Petróleo, Gás e Biocombustíveis',
 'Exploração, Refino e Distribuição']

In [47]:
empresa_df.show()

+-----+----+------------+--------------------+--------------------+
|Papel|Tipo|     Empresa|               Setor|            Subsetor|
+-----+----+------------+--------------------+--------------------+
|PETR4|  PN|PETROBRAS PN|Petróleo, Gás e B...|Exploração, Refin...|
+-----+----+------------+--------------------+--------------------+



In [16]:
# Salvar Spark DataFrame no HDFS
def salvar_spark_dataframe_hdfs(url_hdfs, empresa_df):
    empresa_df.write.save(url_hdfs, format='parquet', mode='append')


### URLs dos dados no HDFS

In [32]:
def retorna_url_hdfs_dados_empresa():
    url_hdfs = 'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/info_empresa'
    
    return url_hdfs

In [33]:
def retorna_url_hdfs_cotacao_empresa():
    url_hdfs = 'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/cotacao_empresa'
    
    return url_hdfs

In [34]:
def retorna_url_hdfs_oscilacoes_empresa():
    url_hdfs = 'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/oscilacoes_empresa'
    
    return url_hdfs

In [35]:
def retorna_url_hdfs_indicacores_fundamentalistas_empresa():
    url_hdfs = 'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/indicadores_empresa'
    
    return url_hdfs

In [36]:
def retorna_url_hdfs_balanco_empresa():
    url_hdfs = 'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/balanco_empresa'
    
    return url_hdfs

### Informações da Empresa

In [40]:
# Salvar dados da empresa no HDFS
def salvar_fundamentus_dados_empresa():
    url_hdfs = retorna_url_hdfs_dados_empresa()
    
    dataframe_label = extrair_info_empresa_label()
    dataframe_dados = extrair_info_empresa_dados()
    
    empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)
    
    salvar_spark_dataframe_hdfs(url_hdfs, empresa_df)
    
salvar_fundamentus_dados_empresa()

In [42]:
from pyspark.sql.utils import AnalysisException

try:
    url_hdfs = retorna_url_hdfs_cotacao_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    empresa_df
except AnalysisException as err:
    print('Diretório não existe!')

Diretório não existe!


In [57]:
url_hdfs = retorna_url_hdfs_dados_empresa()
url_hdfs

'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/'

In [37]:
# Ler dados da empresa no HDFS
def ler_fundamentus_dados_empresa():
    url_hdfs = retorna_url_hdfs_dados_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    
    return empresa_df

In [38]:
empresa_df = ler_fundamentus_dados_empresa()
empresa_df

DataFrame[Papel: string, Tipo: string, Empresa: string, Setor: string, Subsetor: string]

In [39]:
empresa_df.show()

+-----+-----+------------------+--------------------+--------------------+
|Papel| Tipo|           Empresa|               Setor|            Subsetor|
+-----+-----+------------------+--------------------+--------------------+
|PETR4|   PN|      PETROBRAS PN|Petróleo, Gás e B...|Exploração, Refin...|
|ITUB4|PN N1|ITAUUNIBANCO PN N1|Intermediários Fi...|              Bancos|
|GGBR4|PN N1| GERDAU S.A. PN N1|Siderurgia e Meta...|          Siderurgia|
+-----+-----+------------------+--------------------+--------------------+



In [41]:
str(type(empresa_df))

"<class 'pyspark.sql.dataframe.DataFrame'>"

### Selecionando colunas especificas com PySpark

In [48]:
empresa_df.select('Papel').collect()

[Row(Papel='ITUB4')]

In [49]:
empresa_df.select('Papel', 'Tipo').collect()

[Row(Papel='ITUB4', Tipo='PN N1')]

### Filtrando valores em colunas com PySpark

In [70]:
empresa_df.filter("Papel == 'PETR4'")

DataFrame[Papel: string, Tipo: string, Empresa: string, Setor: string, Subsetor: string]

In [73]:
resultado = empresa_df.filter("Papel == 'PETR4'").collect()
resultado

[Row(Papel='PETR4', Tipo='PN', Empresa='PETROBRAS PN', Setor='Petróleo, Gás e Biocombustíveis', Subsetor='Exploração, Refino e Distribuição')]

In [74]:
if (resultado is None) or (len(resultado) == 0):
    print('Vazio')
else:
    print(True)

True


In [60]:
empresa_df.filter("Papel == 'PETR4'").collect()[0]

Row(Papel='PETR4', Tipo='PN', Empresa='PETROBRAS PN', Setor='Petróleo, Gás e Biocombustíveis', Subsetor='Exploração, Refino e Distribuição')

In [61]:
empresa_df.filter("Papel == 'PETR4'").collect()[0]['Papel']

'PETR4'

### Convertendo Spark Dataframe em Pandas Dataframe

In [94]:
empresa_df.toPandas()

,Papel,Tipo,Empresa,Setor,Subsetor
0,PETR4,PN,PETROBRAS PN,"Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
1,ITUB4,PN N1,ITAUUNIBANCO PN N1,Intermediários Financeiros,Bancos
2,GGBR4,PN N1,GERDAU S.A. PN N1,Siderurgia e Metalurgia,Siderurgia


### Cotações da Empresa

In [ ]:
# Salvar cotação da ação da empresa no HDFS
def salvar_fundamentus_cotacao_empresa(papel):
    url_hdfs = retorna_url_hdfs_cotacao_empresa()
    
    dataframe_label = extrair_cotacao_label()
    dataframe_dados = extrair_cotacao_dados(papel)
    
    empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)
    
    salvar_spark_dataframe_hdfs(url_hdfs, empresa_df)
    
salvar_fundamentus_cotacao_empresa(papel)

In [33]:
# Ler cotação da ação da empresa no HDFS
def ler_fundamentus_cotacao_empresa():
    url_hdfs = retorna_url_hdfs_cotacao_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    
    return empresa_df

In [34]:
empresa_df = ler_fundamentus_cotacao_empresa()

In [35]:
type(empresa_df)

pyspark.sql.dataframe.DataFrame

In [36]:
empresa_df.show()

+-----+-------+------------+----------+----------+-------------+----------------+--------------+
|Papel|Cotacao|Data_ult_cot|Min_52_sem|Max_52_sem| Vol_$_med_2m|Valor_de_mercado|     Nro_Acoes|
+-----+-------+------------+----------+----------+-------------+----------------+--------------+
|PETR4|  23,97|  13/04/2021|     15,72|     31,12|2.473.550.000| 312.677.000.000|13.044.500.000|
|ITUB4|  27,29|  13/04/2021|     20,57|     32,51|1.077.330.000| 267.555.000.000| 9.804.140.000|
|GGBR4|  31,94|  13/04/2021|     10,75|     31,94|  496.379.000|  54.925.900.000| 1.719.660.000|
+-----+-------+------------+----------+----------+-------------+----------------+--------------+



In [357]:
empresa_df.select("Papel").collect()

[Row(Papel='PETR4'),
 Row(Papel='VALE3'),
 Row(Papel='ITUB4'),
 Row(Papel='GGBR4')]

In [256]:
empresa_df.filter("Papel == 'PETR4'").show()

+-----+-------+------------+----------+----------+-------------+----------------+--------------+
|Papel|Cotacao|Data_ult_cot|Min_52_sem|Max_52_sem| Vol_$_med_2m|Valor_de_mercado|     Nro_Acoes|
+-----+-------+------------+----------+----------+-------------+----------------+--------------+
|PETR4|  24,04|  05/04/2021|     15,72|     31,12|2.578.990.000| 313.590.000.000|13.044.500.000|
+-----+-------+------------+----------+----------+-------------+----------------+--------------+



In [257]:
len(empresa_df.filter("Papel == 'PETR4'").collect())

1

In [362]:
filterA = "Papel == 'PETR4'"
filterB = "Data_ult_cot == '26/03/2021'"
empresa_df.filter(filterA).filter(filterB).show()

+-----+-------+------------+----------+----------+-------------+----------------+--------------+
|Papel|Cotacao|Data_ult_cot|Min_52_sem|Max_52_sem| Vol_$_med_2m|Valor_de_mercado|     Nro_Acoes|
+-----+-------+------------+----------+----------+-------------+----------------+--------------+
|PETR4|  23,46|  26/03/2021|     13,30|     31,12|2.627.880.000| 306.024.000.000|13.044.500.000|
+-----+-------+------------+----------+----------+-------------+----------------+--------------+



In [108]:
len(empresa_df.filter(filterA).filter(filterB).collect())

0

In [109]:
empresa_df is None

False

In [153]:
papel = "PETR4"
data = "02/03/2021"

In [154]:
filterA = "Papel == '%s'" % papel
filterB = "Data_ult_cot >= '%s'" % data
empresa_df.filter(filterA).filter(filterB).show()

+-----+-------+------------+----------+----------+------------+----------------+---------+
|Papel|Cotacao|Data_ult_cot|Min_52_sem|Max_52_sem|Vol_$_med_2m|Valor_de_mercado|Nro_Acoes|
+-----+-------+------------+----------+----------+------------+----------------+---------+
+-----+-------+------------+----------+----------+------------+----------------+---------+



In [91]:
len(empresa_df.filter(filterA).filter(filterB).collect())

1

### Oscilações Empresa

In [237]:
# Salvar oscilações do papel da empresa no HDFS
def salvar_fundamentus_oscilacoes_empresa(papel):
    url_hdfs = retorna_url_hdfs_oscilacoes_empresa()
    
    dataframe_label = extrair_oscilacao_cotacao_label()
    dataframe_dados = extrair_oscilacao_cotacao_dados(papel)
    
    empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)
    
    salvar_spark_dataframe_hdfs(url_hdfs, empresa_df)
    
salvar_fundamentus_oscilacoes_empresa(papel)

In [37]:
# Ler oscilações do papel da empresa no HDFS
def ler_fundamentus_oscilacoes_empresa():
    url_hdfs = retorna_url_hdfs_oscilacoes_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    
    return empresa_df

In [38]:
empresa_df = ler_fundamentus_oscilacoes_empresa()
empresa_df.show()

+-----+------------+--------------+--------------+------------------+-------------------+---------------+---------------+---------------+---------------+---------------+---------------+
|Papel|Data_ult_cot|Oscilacoes_Dia|Oscilacoes_Mes|Oscilacoes_30_dias|Oscilacoes_12_meses|Oscilacoes_2021|Oscilacoes_2020|Oscilacoes_2019|Oscilacoes_2018|Oscilacoes_2017|Oscilacoes_2016|
+-----+------------+--------------+--------------+------------------+-------------------+---------------+---------------+---------------+---------------+---------------+---------------+
|ITUB4|  13/04/2021|        -0,04%|        -2,37%|             0,62%|             15,45%|        -12,88%|        -11,75%|         11,98%|         32,41%|         29,64%|         56,02%|
|GGBR4|  13/04/2021|         2,31%|         5,69%|            16,15%|            188,81%|         31,15%|         23,86%|         37,65%|         22,31%|         15,23%|        138,66%|
|PETR4|  13/04/2021|         0,33%|        -0,54%|             3,45%| 

In [240]:
empresa_df.toPandas()

,Papel,Data_ult_cot,Oscilacoes_Dia,Oscilacoes_Mes,Oscilacoes_30_dias,Oscilacoes_12_meses,Oscilacoes_2021,Oscilacoes_2020,Oscilacoes_2019,Oscilacoes_2018,Oscilacoes_2017,Oscilacoes_2016
0,MGLU3,05/04/2021,"-0,10%","0,89%","-18,74%","130,62%","-18,16%","110,08%","112,19%","126,39%","509,27%","503,87%"
1,ITUB4,05/04/2021,"1,29%","-1,80%","-0,44%","34,90%","-12,37%","-11,75%","11,98%","32,41%","29,64%","56,02%"
2,PETR4,05/04/2021,"0,63%","-0,25%","7,37%","56,71%","-15,17%","-6,10%","36,87%","45,91%","8,29%","116,23%"
3,VALE3,05/04/2021,"6,16%","5,53%","3,17%","176,82%","23,39%","70,47%","6,85%","31,11%","62,56%","103,66%"


### Indicadores Fundamentalistas

In [241]:
# Salvar indicadores fundamentalistas da empresa no HDFS
def salvar_fundamentus_indicacores_fundamentalistas_empresa(papel):
    url_hdfs = retorna_url_hdfs_indicacores_fundamentalistas_empresa()
    
    dataframe_label = extrair_indicadores_fundamentalistas_label()
    dataframe_dados = extrair_indicadores_fundamentalistas_dados(papel)
    
    
    empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)
    
    salvar_spark_dataframe_hdfs(url_hdfs, empresa_df)
    
salvar_fundamentus_indicacores_fundamentalistas_empresa(papel)

In [87]:
# Ler indicadores fundamentalistas da empresa no HDFS
def ler_fundamentus_indicacores_fundamentalistas_empresa():
    url_hdfs = retorna_url_hdfs_indicacores_fundamentalistas_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    
    return empresa_df

In [88]:
empresa_df = ler_fundamentus_indicacores_fundamentalistas_empresa()
empresa_df.show()

+-----+------------+-----+----+----+-----+------+----------+----+---------+--------+------------+----------+------------+---------------+-----+---------+----+-----------+-------------+---------+--------------+-----------+-----------+
|Papel|Data_ult_cot|  P/L| LPA|P/VP|  VPA|P/EBIT|Marg_Bruta| PSR|Marg_EBIT|P/Ativos|Marg_Liquida|P/Cap_Giro|EBIT_/_Ativo|P/Ativ_Circ_Liq| ROIC|Div_Yield| ROE|EV_/_EBITDA|Liquidez_Corr|EV_/_EBIT|Div_Br/_Patrim|Cres_Rec_5a|Giro_Ativos|
+-----+------------+-----+----+----+-----+------+----------+----+---------+--------+------------+----------+------------+---------------+-----+---------+----+-----------+-------------+---------+--------------+-----------+-----------+
|PETR4|  13/04/2021|43,99|0,54|1,01|23,64|  3,35|     45,6%|1,15|    34,3%|    0,32|        2,3%|     51,80|        9,5%|          -0,59|10,5%|     0,0%|2,3%|       4,09|         1,04|     6,86|          1,27|      -0,1%|       0,28|
|GGBR4|  13/04/2021|23,22|1,38|1,78|17,95| 12,48|     13,5%|1,25

In [244]:
empresa_df.toPandas()

,Papel,Data_ult_cot,P/L,LPA,P/VP,VPA,P/EBIT,Marg_Bruta,PSR,Marg_EBIT,...,P/Ativ_Circ_Liq,ROIC,Div_Yield,ROE,EV_/_EBITDA,Liquidez_Corr,EV_/_EBIT,Div_Br/_Patrim,Cres_Rec_5a,Giro_Ativos
0,MGLU3,05/04/2021,"338,79","0,06","18,12","1,13","177,41","25,8%","4,55","2,6%",...,"-253,69","5,6%","0,2%","5,3%","90,65","1,25","175,79","0,23","31,6%","1,18"
1,VALE3,05/04/2021,"20,45","5,05","2,94","35,16","5,10","52,7%","2,62","51,4%",...,"-3,21","27,4%","6,5%","14,4%","4,48","1,67","5,18","0,42","20,9%","0,44"
2,PETR4,05/04/2021,"44,12","0,54","1,02","23,64","3,36","45,6%","1,15","34,3%",...,"-0,59","10,5%","0,0%","2,3%","4,09","1,04","6,87","1,27","-0,1%","0,28"
3,ITUB4,05/04/2021,"14,19","1,93","1,97","13,94",-,-,-,-,...,-,-,"2,0%","13,9%",-,-,-,-,"-30,0%",-


### Balanço Empresa

In [245]:
# Salvar balanço da empresa no HDFS
def salvar_fundamentus_balanco_empresa(papel):
    url_hdfs = retorna_url_hdfs_balanco_empresa()
    
    dataframe_label = extrair_balanco_label()
    dataframe_dados = extrair_balanco_dados(papel)
    
    empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)
    
    salvar_spark_dataframe_hdfs(url_hdfs, empresa_df)
    
salvar_fundamentus_balanco_empresa(papel)

In [39]:
# Ler balanço da empresa no HDFS
def ler_fundamentus_balanco_empresa():
    url_hdfs = retorna_url_hdfs_balanco_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    
    return empresa_df

In [40]:
empresa_df = ler_fundamentus_balanco_empresa()
empresa_df.show()

+-----+----------------------+---------------+---------------+---------------+----------------+---------------+----------------+---------------+----------------------------+-----------------+--------------------------+---------------------------+----------------+-------------------------+
|Papel|Ult_balanco_processado| Valor_da_firma|          Ativo|      Div_Bruta|Disponibilidades|    Div_Liquida|Ativo_Circulante|     Patrim_Liq|Receita_Liquida_ult_12_meses|EBIT_ult_12_meses|Lucro_Liquido_ult_12_meses|Receita_Liquida_ult_3_meses|EBIT_ult_3_meses|Lucro_Liquido_ult_3_meses|
+-----+----------------------+---------------+---------------+---------------+----------------+---------------+----------------+---------------+----------------------------+-----------------+--------------------------+---------------------------+----------------+-------------------------+
|PETR4|            31/12/2020|640.945.000.000|987.419.000.000|392.548.000.000|  64.280.000.000|328.268.000.000| 142.323.000.000|30

In [250]:
str(type(empresa_df))

"<class 'pyspark.sql.dataframe.DataFrame'>"

In [2]:
#sparkSession.createDataFrame(info_empresa_dados).collect()

In [3]:
#sparkSession.createDataFrame(info_empresa_dados, info_empresa_label).collect()

In [4]:
#df.show()

## Write Spark DataFrame dos Dados de Cotação

In [ ]:
cotacao_label.insert(0, 'Papel')
cotacao_dados.insert(0, papel)
cotacao_dados = transformar_lista_em_tuples(cotacao_dados)
cotacao_dados

In [51]:
df = sparkSession.createDataFrame(cotacao_dados, cotacao_label)
df

DataFrame[Papel: string, Cotacao: string, Data_ult_cot: string, Min_52_sem: string, Max_52_sem: string, Vol_$_med_2m: string, Valor_de_mercado: string, Nro._Acoes: string]

In [52]:
df.write.save('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_cotacao', format='parquet', mode='append')

In [53]:
df_load = sparkSession.read.format('parquet').load('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_cotacao')

In [54]:
df_load.show()

+-----+-------+------------+----------+----------+-------------+----------------+--------------+
|Papel|Cotacao|Data_ult_cot|Min_52_sem|Max_52_sem| Vol_$_med_2m|Valor_de_mercado|    Nro._Acoes|
+-----+-------+------------+----------+----------+-------------+----------------+--------------+
|VALE3| 101,60|  02/03/2021|     32,82|    102,32|3.111.610.000| 536.903.000.000| 5.284.470.000|
|PETR4|  21,99|  02/03/2021|     11,29|     31,12|2.600.720.000| 286.848.000.000|13.044.500.000|
|ITUB4|  25,77|  02/03/2021|     20,24|     32,58|1.217.880.000| 252.653.000.000| 9.804.140.000|
|GGBR4|  26,67|  01/03/2021|      8,54|     29,54|  419.645.000|  45.863.300.000| 1.719.660.000|
+-----+-------+------------+----------+----------+-------------+----------------+--------------+



In [55]:
df_load.toPandas()

,Papel,Cotacao,Data_ult_cot,Min_52_sem,Max_52_sem,Vol_$_med_2m,Valor_de_mercado,Nro._Acoes
0,VALE3,"101,60",02/03/2021,"32,82","102,32",3.111.610.000,536.903.000.000,5.284.470.000
1,PETR4,"21,99",02/03/2021,"11,29","31,12",2.600.720.000,286.848.000.000,13.044.500.000
2,ITUB4,"25,77",02/03/2021,"20,24","32,58",1.217.880.000,252.653.000.000,9.804.140.000
3,GGBR4,"26,67",01/03/2021,"8,54","29,54",419.645.000,45.863.300.000,1.719.660.000


## Write Spark DataFrame dos Dados de Oscilações

In [ ]:
oscilacoes_label.insert(0, 'Papel')
oscilacoes_dados.insert(0, papel)
oscilacoes_dados = transformar_lista_em_tuples(oscilacoes_dados)
oscilacoes_dados

In [56]:
df = sparkSession.createDataFrame(oscilacoes_dados, oscilacoes_label)
df

DataFrame[Papel: string, Oscilacoes_Dia: string, Oscilacoes_Mes: string, Oscilacoes_30_dias: string, Oscilacoes_12_meses: string, Oscilacoes_2021: string, Oscilacoes_2020: string, Oscilacoes_2019: string, Oscilacoes_2018: string, Oscilacoes_2017: string, Oscilacoes_2016: string]

In [57]:
df.write.save('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_oscilacoes', format='parquet', mode='append')

In [58]:
df_load = sparkSession.read.format('parquet').load('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_oscilacoes')

In [59]:
df_load.show()

+-----+--------------+--------------+------------------+-------------------+---------------+---------------+---------------+---------------+---------------+---------------+
|Papel|Oscilacoes_Dia|Oscilacoes_Mes|Oscilacoes_30_dias|Oscilacoes_12_meses|Oscilacoes_2021|Oscilacoes_2020|Oscilacoes_2019|Oscilacoes_2018|Oscilacoes_2017|Oscilacoes_2016|
+-----+--------------+--------------+------------------+-------------------+---------------+---------------+---------------+---------------+---------------+---------------+
|PETR4|        -0,05%|        -1,12%|           -23,27%|            -17,11%|        -22,41%|         -6,10%|         36,87%|         45,91%|          8,29%|        116,23%|
|ITUB4|         4,04%|         0,93%|            -8,90%|            -18,04%|        -17,91%|        -11,75%|         11,98%|         32,41%|         29,64%|         56,02%|
|VALE3|         3,07%|         7,49%|            15,90%|            127,71%|         16,18%|         70,47%|          6,85%|         31

In [60]:
df_load.toPandas()

,Papel,Oscilacoes_Dia,Oscilacoes_Mes,Oscilacoes_30_dias,Oscilacoes_12_meses,Oscilacoes_2021,Oscilacoes_2020,Oscilacoes_2019,Oscilacoes_2018,Oscilacoes_2017,Oscilacoes_2016
0,PETR4,"-0,05%","-1,12%","-23,27%","-17,11%","-22,41%","-6,10%","36,87%","45,91%","8,29%","116,23%"
1,ITUB4,"4,04%","0,93%","-8,90%","-18,04%","-17,91%","-11,75%","11,98%","32,41%","29,64%","56,02%"
2,VALE3,"3,07%","7,49%","15,90%","127,71%","16,18%","70,47%","6,85%","31,11%","62,56%","103,66%"
3,GGBR4,"3,09%","3,09%","16,11%","55,83%","9,08%","23,86%","37,65%","22,31%","15,23%","138,66%"


## Write Spark DataFrame dos Dados de Indicacores Fundamentalistas

In [ ]:
indicador_fundamentalita_label.insert(0, "Papel")
indicador_fundamentalita_dados.insert(0, papel)
indicador_fundamentalita_dados = transformar_lista_em_tuples(indicador_fundamentalita_dados)
indicador_fundamentalita_dados

In [61]:
df = sparkSession.createDataFrame(indicador_fundamentalita_dados, indicador_fundamentalita_label)
df

DataFrame[Papel: string, P/L: string, LPA: string, P/VP: string, VPA: string, P/EBIT: string, Marg._Bruta: string, PSR: string, Marg._EBIT: string, P/Ativos: string, Marg._Liquida: string, P/Cap._Giro: string, EBIT_/_Ativo: string, P/Ativ_Circ_Liq: string, ROIC: string, Div._Yield: string, ROE: string, EV_/_EBITDA: string, Liquidez_Corr: string, EV_/_EBIT: string, Div_Br/_Patrim: string, Cres._Rec_5a: string, Giro_Ativos: string]

In [62]:
df.write.save('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_indicador_fundamentalista', format='parquet', mode='append')

In [63]:
df_load = sparkSession.read.format('parquet').load('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_indicador_fundamentalista')

In [64]:
df_load.show()

+-----+-----+----+----+-----+------+-----------+----+----------+--------+-------------+-----------+------------+---------------+-----+----------+-----+-----------+-------------+---------+--------------+------------+-----------+
|Papel|  P/L| LPA|P/VP|  VPA|P/EBIT|Marg._Bruta| PSR|Marg._EBIT|P/Ativos|Marg._Liquida|P/Cap._Giro|EBIT_/_Ativo|P/Ativ_Circ_Liq| ROIC|Div._Yield|  ROE|EV_/_EBITDA|Liquidez_Corr|EV_/_EBIT|Div_Br/_Patrim|Cres._Rec_5a|Giro_Ativos|
+-----+-----+----+----+-----+------+-----------+----+----------+--------+-------------+-----------+------------+---------------+-----+----------+-----+-----------+-------------+---------+--------------+------------+-----------+
|VALE3|20,10|5,05|2,89|35,16|  5,01|      52,7%|2,57|     51,4%|    1,12|        11,9%|      10,53|       22,4%|          -3,15|27,4%|      2,4%|14,4%|       4,40|         1,67|     5,09|          0,42|       20,9%|       0,44|
|PETR4|40,36|0,54|0,93|23,64|  3,07|      45,6%|1,05|     34,3%|    0,29|         2,3%| 

In [65]:
df_load.toPandas()

,Papel,P/L,LPA,P/VP,VPA,P/EBIT,Marg._Bruta,PSR,Marg._EBIT,P/Ativos,...,P/Ativ_Circ_Liq,ROIC,Div._Yield,ROE,EV_/_EBITDA,Liquidez_Corr,EV_/_EBIT,Div_Br/_Patrim,Cres._Rec_5a,Giro_Ativos
0,VALE3,"20,10","5,05","2,89","35,16","5,01","52,7%","2,57","51,4%","1,12",...,"-3,15","27,4%","2,4%","14,4%","4,40","1,67","5,09","0,42","20,9%","0,44"
1,PETR4,"40,36","0,54","0,93","23,64","3,07","45,6%","1,05","34,3%","0,29",...,"-0,54","10,5%","0,0%","2,3%","3,92","1,04","6,58","1,27","-0,1%","0,28"
2,GGBR4,"19,39","1,38","1,49","17,95","10,42","13,5%","1,05","10,0%","0,73",...,"-5,32","8,8%","1,1%","7,7%","8,14","2,04","12,66","0,57","3,8%","0,69"
3,ITUB4,"13,32","1,93","1,85","13,94",-,-,-,-,-,...,-,-,"1,9%","13,9%",-,-,-,-,"-30,0%",-


## Write Spark DataFrame dos Dados do Balanço

In [ ]:
balanco_label.insert(0, "Papel")
balanco_dados.insert(0, papel)
balanco_dados = transformar_lista_em_tuples(balanco_dados)
balanco_dados

In [66]:
df = sparkSession.createDataFrame(balanco_dados, balanco_label)
df

DataFrame[Papel: string, Ult_balanco_processado: string, Valor_da_firma: string, Ativo: string, Div._Bruta: string, Disponibilidades: string, Div._Liquida: string, Ativo_Circulante: string, Patrim._Liq: string, Receita_Liquida_ult._12_meses: string, EBIT_ult._12_meses: string, Lucro_Liquido_ult._12_meses: string, Receita_Liquida_ult._3_meses: string, EBIT_ult._3_meses: string, Lucro_Liquido_ult._3_meses: string]

In [67]:
df.write.save('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_balanco', format='parquet', mode='append')

In [68]:
df_load = sparkSession.read.format('parquet').load('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_balanco')

In [69]:
df_load.show()

+-----+----------------------+---------------+---------------+---------------+----------------+---------------+----------------+---------------+-----------------------------+------------------+---------------------------+----------------------------+-----------------+--------------------------+
|Papel|Ult_balanco_processado| Valor_da_firma|          Ativo|     Div._Bruta|Disponibilidades|   Div._Liquida|Ativo_Circulante|    Patrim._Liq|Receita_Liquida_ult._12_meses|EBIT_ult._12_meses|Lucro_Liquido_ult._12_meses|Receita_Liquida_ult._3_meses|EBIT_ult._3_meses|Lucro_Liquido_ult._3_meses|
+-----+----------------------+---------------+---------------+---------------+----------------+---------------+----------------+---------------+-----------------------------+------------------+---------------------------+----------------------------+-----------------+--------------------------+
|PETR4|            31/12/2020|615.116.000.000|987.419.000.000|392.548.000.000|  64.280.000.000|328.268.000.000| 

In [70]:
df_load.toPandas()

,Papel,Ult_balanco_processado,Valor_da_firma,Ativo,Div._Bruta,Disponibilidades,Div._Liquida,Ativo_Circulante,Patrim._Liq,Receita_Liquida_ult._12_meses,EBIT_ult._12_meses,Lucro_Liquido_ult._12_meses,Receita_Liquida_ult._3_meses,EBIT_ult._3_meses,Lucro_Liquido_ult._3_meses
0,PETR4,31/12/2020,615.116.000.000,987.419.000.000,392.548.000.000,64.280.000.000,328.268.000.000,142.323.000.000,308.410.000.000,272.069.000.000,93.417.000.000,7.108.000.000,74.972.000.000,33.838.000.000,59.890.000.000
1,VALE3,31/12/2020,544.906.000.000,478.129.000.000,78.089.100.000,70.085.600.000,8.003.530.000,126.805.000.000,185.785.000.000,208.529.000.000,107.104.000.000,26.712.700.000,78.938.200.000,47.152.400.000,4.824.830.000
2,GGBR4,31/12/2020,55.720.300.000,63.123.000.000,17.515.400.000,7.658.350.000,9.857.000.000,23.409.500.000,30.860.300.000,43.814.700.000,4.400.170.000,2.365.760.000,13.620.200.000,2.176.850.000,1.048.330.000
3,ITUB4,31/12/2020,-,210.399.000.000,None,None,None,None,136.699.000.000,None,None,18.961.000.000,None,None,7.636.000.000


In [154]:
class TesteStatic:
    
    def __init__(self):
        self.__numero = {}
    

    def metodo_static1(self):
        self.__numero['metodo1'] = 1
        self.metodo_static2()
        return self.__numero
    

    def metodo_static2(self):
        self.__numero['metodo2'] = 2
        return self.__numero 
    

    def metodo_static3(self):
        self.__numero['metodo3'] = 3
        return self.__numero 

    
    
    

In [161]:
teste = TesteStatic()
teste

In [166]:
print(teste.metodo_static1())

{'metodo2': 2, 'metodo1': 1, 'metodo3': 3}


In [1]:
teste = {'nome':'hugo', 'idade':31}

In [2]:
dados = {}
dados['Pessoa'] = teste

In [6]:
for i in dados:
    print(i)

Pessoa


In [9]:
dados['Pessoa']['nome']

'hugo'